# Real time signal processing in EEG

## Import libraries
To install libraries in Python, you can use a package manager like pip, which comes pre-installed with most Python distributions.

In [1]:
import pandas as pd
from scipy import signal
import numpy as np
import time
import pylab as pl
from IPython import display

## Real time signal processing in EEG

Real-time signal processing in EEG (electroencephalography) is critical for several reasons. First, it provides immediate analysis and interpretation of brain activity, which is important for applications such as neurofeedback, brain-computer interfaces (BCI), and real-time monitoring of neurological conditions.

In addition, real-time processing facilitates adaptive algorithms that can dynamically adapt to changing brain states, increasing the accuracy and effectiveness of interventions. This immediacy also improves the user experience in BCI by reducing latency, resulting in a smoother, more responsive experience.

**Positive**  

* Responsiveness: Real-time processing minimizes latency, making BCIs more effective and user-friendly by providing quick responses to user inputs.  
* Adaptive Algorithms: Algorithms can dynamically adjust to ongoing changes in brain activity, enhancing the accuracy and functionality of BCIs.

**Negative**  

* Signal Noise: EEG signals are often contaminated with noise from various sources (e.g., muscle movements, electrical interference), which can complicate real-time processing and reduce accuracy.  
* Computational Load: Real-time processing requires significant computational resources to filter, analyze, and interpret EEG data promptly.  
* Accuracy: The need for fast processing can sometimes compromise the accuracy of signal interpretation, leading to false positives or negatives.  
* Generalization: Real-time algorithms may not generalize well across different subjects or conditions, necessitating extensive customization and tuning.  

**How it works**

When working in real time, the main difference is the need to create a data buffer. It's crucial to determine the length of data that will be fed into the filter for processing and subsequent visualization. For instance, with a frame rate of 250 FPS (meaning we receive 250 EEG data points from each channel per second), data processing and visualization would occur once per secund. However, for bandpass filters, it is important to supply data with a margin, as the filter needs time for adjusting.

In our example, we demonstrate the ability to set the length of one data packet and how many packets will be fed into the filter.

**Recommendation**

Try adjusting the bandpass filter settings and aim to achieve a faster graph update rate. Currently, the graph updates every sec.
